In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [9]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

LENGTH = 2648
RECS = 1384 * 9
TRANSL = 307

X = np.zeros([RECS, LENGTH, 20], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(9):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()


12429 12456


In [ ]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        self.fname = fname
        super(MyCallback, self).__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {}\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

logger = MyCallback("./training-bigger-log.txt")
            
def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    layer = Lambda(lambda x: (x - MEAN) / STD)(feature_input)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_1 = Conv1D(512, 5, activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    for i in range(7):
        layer = Bidirectional(GRU(256, return_sequences = True, recurrent_dropout=0.01, kernel_initializer=Orthogonal(), activation='linear'))(layer)
        layer = LeakyReLU(0.01)(layer)
        layer = BatchNormalization()(layer)
    layer = Dense(1024)(layer)
    layer = LeakyReLU(0.01)(layer)
    layer = BatchNormalization()(layer)
    layer_15 = Dense(NPHONES + 1, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX, trainy, trainXl, trainyl], np.zeros(trainX.shape[0]), epochs = 100,
                     batch_size = 32, callbacks=[mc])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=1000) # at 300 it makes sensible predictions
    valid_data = make_data(*load_data(VALIDDATA))
    predictions = predict.predict(valid_data[0])
    DERs = validate(predictions, valid_data[2], valid_data[1], valid_data[3])
    print("Validation Digit Error Rate: {}".format(sum([x[0] * x[1] for x in DERs]) / sum([x[1] for x in DERs])))
    predict.save('cyfry.pred.h5')

[(12416, 2648, 20), (12416, 307), (12416, 1), (12416, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, None, 20)     0           input_7[0][0]                    
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, None, 20, 1)  0           lambda_9[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, None, 20, 12) 72          lambda_10[0][0]                  
___________________________________________________

Epoch 1/100
12416/12416 [==============================] - 3788s 305ms/step - loss: 477.7933
Epoch 2/100
12416/12416 [==============================] - 3776s 304ms/step - loss: 342.6235
Epoch 3/100
12416/12416 [==============================] - 3776s 304ms/step - loss: 239.9440
Epoch 4/100
12416/12416 [==============================] - 3778s 304ms/step - loss: 204.3700
Epoch 5/100
12416/12416 [==============================] - 3782s 305ms/step - loss: 184.1607
Epoch 6/100
12416/12416 [==============================] - 3781s 304ms/step - loss: 169.9461
Epoch 7/100
12416/12416 [==============================] - 3779s 304ms/step - loss: 159.2866
Epoch 8/100
12416/12416 [==============================] - 3780s 304ms/step - loss: 150.5535
Epoch 9/100
12416/12416 [==============================] - 3784s 305ms/step - loss: 142.5209
Epoch 10/100
12416/12416 [==============================] - 3780s 304ms/step - loss: 135.9331
Epoch 11/100
12416/12416 [==============================] - 3782s 305

In [13]:
GRU(np.zeros(NPHONES + 1))(Input((30,10)))

TypeError: only size-1 arrays can be converted to Python scalars

In [11]:
pr = predict.predict(X[:1])
pr = pr.argmax(2)[0]
pr

array([20, 20, 20, 20, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37,  3, 37, 31, 37, 37, 28, 28, 28, 37,  4, 37, 37, 29, 20, 28, 37,
       37, 34, 37, 37, 31, 37, 21, 37, 37, 34, 34, 37,  4, 37,  6, 20, 34,
       37,  8, 37, 37, 28, 37, 37, 15, 37, 24, 33, 33, 14, 14, 37, 29, 37,
       23, 37, 37,  8, 37, 10, 20,  8, 37, 37, 34, 37,  0,  0, 37,  6, 37,
       34, 37, 37, 37, 33, 37, 37,  0, 37, 37, 30, 37, 37,  6, 37, 37, 37,
       37, 37, 37, 20, 20, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 31, 37,
       20, 28, 37, 37, 28, 37, 37, 37, 29, 37, 28, 37, 37, 37,  6, 37, 10,
       37, 37, 27, 27, 37, 37,  3,  3, 37, 37, 29, 37, 30, 37, 37,  0, 37,
       37, 37, 37, 12, 37,  6, 37, 37, 37,  5, 37, 37, 22, 37,  6, 37, 28,
        4,  4,  4, 37, 37

In [12]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, [271])[0][0]
pr

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


<tf.Tensor 'SparseToDense:0' shape=(1, 63) dtype=int64>

In [13]:
pred_tr = pr.eval(session=K.get_session())

In [14]:
Y[:1]

array([[20,  3, 31, 28,  4, 29, 20, 28, 34, 31, 21,  8, 34,  0,  6, 20,
        34,  8, 28, 15, 24, 33, 14, 29, 23,  8, 10, 20,  8, 34, 20,  0,
         6, 34, 33, 20,  0, 30,  6, 20, 31, 20, 28, 28, 29, 28,  6, 20,
        10, 27,  3, 29, 30,  0, 20, 12, 29,  5, 22,  6, 20, 28,  4, 21,
         6,  4, 20, 31, 36, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [15]:
import editdistance
editdistance.eval(list(pred_tr[0]), list(Y[0, :70]))

9

In [16]:
pred_tr[0]

array([20,  3, 31, 28,  4, 29, 20, 28, 34, 31, 21, 34,  4,  6, 20, 34,  8,
       28, 15, 24, 33, 14, 29, 23,  8, 10, 20,  8, 34,  0,  6, 34, 33,  0,
       30,  6, 20, 31, 20, 28, 28, 29, 28,  6, 10, 27,  3, 29, 30,  0, 12,
        6,  5, 22,  6, 28,  4, 21,  6,  4, 31, 36, 20])

In [17]:
preds = predict.predict(X[:160])

In [18]:
pr = K.ctc_decode(preds, X_lens[:160])[0][0]

In [19]:
pr.eval(session=K.get_session())

array([[20,  3, 31, ..., -1, -1, -1],
       [20, 24, 22, ..., -1, -1, -1],
       [20, 10,  2, ..., -1, -1, -1],
       ...,
       [20, 18, 20, ..., -1, -1, -1],
       [20, 22,  6, ..., -1, -1, -1],
       [20, 28, 31, ..., -1, -1, -1]])

In [21]:
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]

In [22]:
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]

In [23]:
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.12857142857142856,
 0.06976744186046512,
 0.04838709677419355,
 0.0875,
 0.09917355371900827,
 0.031914893617021274,
 0.18518518518518517,
 0.1,
 0.04411764705882353,
 0.043478260869565216,
 0.13924050632911392,
 0.05555555555555555,
 0.07692307692307693,
 0.23529411764705882,
 0.07692307692307693,
 0.23529411764705882,
 0.14473684210526316,
 0.24390243902439024,
 0.11235955056179775,
 0.1686746987951807,
 0.05357142857142857,
 0.023255813953488372,
 0.08247422680412371,
 0.13725490196078433,
 0.08653846153846154,
 0.06930693069306931,
 0.05194805194805195,
 0.09090909090909091,
 0.09722222222222222,
 0.06060606060606061,
 0.0898876404494382,
 0.06944444444444445,
 0.012345679012345678,
 0.1791044776119403,
 0.09375,
 0.038461538461538464,
 0.10843373493975904,
 0.10126582278481013,
 0.09302325581395349,
 0.05747126436781609,
 0.06172839506172839,
 0.05952380952380952,
 0.04819277108433735,
 0.11494252873563218,
 0.13186813186813187,
 0.15217391304347827,
 0.11267605633802817,
 0.12

In [20]:
hypos

NameError: name 'hypos' is not defined

In [24]:
trn.evaluate(list(data), [np.zeros(len(data[0]))])

12416/12416 [==============================] - 615s 50ms/step


29.3532679695444

In [1]:
## 


NameError: name 'trn' is not defined